In [10]:
import numpy as np
from PIL import Image
import torch
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity as lpips
from torchvision import transforms
import matplotlib.pyplot as plt


In [4]:
epsilon = 16/255
alpha = 1.6/255
epochs = 10
decay = 1

lpips_loss = lpips(net_type='squeeze', normalize=True)

Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to C:\Users\Kshat/.cache\torch\hub\checkpoints\squeezenet1_1-b8a52dc0.pth
100.0%


In [5]:
def add_noise(x, noise=None):
        if noise != None:
               return torch.clip(x + noise, 0, 1)
        return torch.clip(x + torch.zeros_like(x).uniform_(-16/255,16/255), 0, 1)

def tensor_to_image(image):
      img = image.permute(1, 2, 0).cpu().detach().numpy() * 255
      img = img.astype(np.uint8)
      img = Image.fromarray(img)
      return img

def init_delta(data):
        delta = torch.zeros_like(data)
        delta.uniform_(-epsilon, epsilon)
        delta.requires_grad = True
        return delta

def get_loss(image1, image2):
       res = lpips_loss(image1, image2)
       return res

def update_delta(delta, grad):
      delta = torch.clamp(delta + alpha * grad.sign(), -epsilon, epsilon)
      return delta
       

image_path = "dog.jpg"

transform = transforms.Compose([
  transforms.ToTensor()  # Convert to tensor with pixel values between 0 and 1
])

In [6]:
image = Image.open(image_path).convert('RGB')

transformed_image = transform(image)
transformed_image.requires_grad = True

transformed_image = transformed_image.view(1,3,1000,1500)

In [17]:
delta = init_delta(transformed_image)
noised_image = add_noise(transformed_image, delta)
noised_image = noised_image.view(1,3,1000,1500)

momentum = 0.
for _ in range(10):
        loss = get_loss(transformed_image, noised_image)
        grad = torch.autograd.grad(loss, delta)[0]
        momentum = momentum * decay + grad / (grad.abs().mean(dim=(1,2,3), keepdim=True))
        delta = update_delta(delta, momentum)
        noised_image = add_noise(noised_image, delta)
        new_img = tensor_to_image(noised_image[0])
        new_img.show()
        new_img.save(f'LPIPS_Images/Lpips_image_{_}.jpg')
        print(loss)

tensor(0.4750, grad_fn=<SqueezeBackward0>)
tensor(0.7252, grad_fn=<SqueezeBackward0>)
tensor(0.8424, grad_fn=<SqueezeBackward0>)
tensor(0.9247, grad_fn=<SqueezeBackward0>)
tensor(0.9829, grad_fn=<SqueezeBackward0>)
tensor(1.0243, grad_fn=<SqueezeBackward0>)
tensor(1.0572, grad_fn=<SqueezeBackward0>)
tensor(1.0849, grad_fn=<SqueezeBackward0>)
tensor(1.1092, grad_fn=<SqueezeBackward0>)
tensor(1.1307, grad_fn=<SqueezeBackward0>)
